In [1]:
import cv2
import numpy as np
import math

vid = cv2.VideoCapture(0) #goruntu alma

#sonsuz bir while dongusu olusturacağız görüntüyü almak için 
#eğer bir sey bulamazsak diye try ve except bloğu oluşturacağız
while(1):
        
    try:  
          
        ret, frame = vid.read()#her bir frame okuyorum
        frame=cv2.flip(frame,1)#ters cevir
        kernel = np.ones((3,3),np.uint8)
        
        roi=frame[100:300, 100:300] #hareketlerin belirleneceği bolum
        
        
        cv2.rectangle(frame,(100,100),(300,300),(0,255,0),0) #kontur   
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)#hsv formatına donusturur
        
        
        #deri rengini belirlemek için bir alt değer bir de ust değer olacak bunu lower_skin ve upper_skin ile hallettik
        lower_skin = np.array([0,20,70], dtype=np.uint8)
        upper_skin = np.array([20,255,255], dtype=np.uint8)
        
        #hsv formatına cevirilmis roiye mask işlemleri uygulayarak el hareketlerimi arka plandan ayıracxağım
        mask = cv2.inRange(hsv, lower_skin, upper_skin)
        
   
        #olusan karanlık yerleri beyaza cevirme
        mask = cv2.dilate(mask,kernel,iterations = 4)
        
        #oluşan görültüleri silmek için gaus yontemi kullandik
        mask = cv2.GaussianBlur(mask,(5,5),100) 
        
        #mask kullanarak sınır çizgilerini bulacağız
        contours,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
        #konturların max alanını belirleyeceğiz
        cnt = max(contours, key = lambda x: cv2.contourArea(x))
        
        #konturlara daha yakın olmamızı sağlıyor
        epsilon = 0.0005*cv2.arcLength(cnt,True)
        approx= cv2.approxPolyDP(cnt,epsilon,True)
       
        #elimizin üstüne dışbükey bir görüntü oluşturuyoruz
        hull = cv2.convexHull(cnt)
        
        #hul alan
        areaHull = cv2.contourArea(hull)
        #elimin alanı 
        areaCnt = cv2.contourArea(cnt)
          
        #konturun içerisinde elimizin olmadığı alanın hesabi 
        areaRatio=((areaHull-areaCnt)/areaCnt)*100
        
        #konturların indislerine erişeceğiz
        hull = cv2.convexHull(approx, returnPoints=False)
        #konturların kusurlarını tutan değişken
        defects = cv2.convexityDefects(approx, hull)
        
        #toplam kusur sayısına şimdilik 0 verelim
        l=0
        
        
        #defects içersindeki kusurların değerlerini tek tek değişkenlere atayacağız çünkü çizim yapacağız
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]#başlangıç ve bitiş noktalarını değişkenlere atadık
            start = tuple(approx[s][0])#başlangıç
            end = tuple(approx[e][0])#bitiş
            far = tuple(approx[f][0])#aralık
            
            
            #hesaplayıp değerleri atadık
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            s = (a+b+c)/2#alan bulmak için kullanıcaz
            ar = math.sqrt(s*(s-a)*(s-b)*(s-c))#alan bulduk
            
            d=(2*ar)/a
            
            #iki kenar arasındaki açı (Kosünüs teoremi uyguladık)
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
            
            
            #eğer açı 90 dereceye eşit veya 30 dereceden büyük ise
            if angle <= 90 and d>30:
                l += 1 #toplam kusur sayısını bir arttır
                cv2.circle(roi, far, 3, [255,0,0], -1)
            
            cv2.line(roi,start, end, [0,255,0], 2)
            
            
        l+=1
        #yazı fontu
        font = cv2.FONT_HERSHEY_SIMPLEX
        #eğer kusur sayısı 1 ise
        if l==1:
            if areaCnt<2000:
                cv2.putText(frame,'Put your hand in the box',(0,50), font, 1, (0,0,255), 3, cv2.LINE_AA)
            else:
                if areaRatio<12:
                    cv2.putText(frame,'0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                elif areaRatio<17.5:
                    cv2.putText(frame,'Best luck',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                   
                else:
                    cv2.putText(frame,'1',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
       #eğer kusur sayısı 2 ise             
        elif l==2:
            cv2.putText(frame,'2',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        #eğer kusur sayısı 3 ise    
        elif l==3:
         
              if areaRatio<27:
                    cv2.putText(frame,'3',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
              else:
                    cv2.putText(frame,'ok',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        #eğer kusur sayısı 4 ise            
        elif l==4:
            cv2.putText(frame,'4',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        #eğer kusur sayısı 5 ise    
        elif l==5:
            cv2.putText(frame,'5',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        #eğer kusur sayısı 6 ise    
        elif l==6:
            cv2.putText(frame,'reposition',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        #eğer kusur sayısı 6 dan farklı bir sayı ise 
        else :
            cv2.putText(frame,'reposition',(10,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
       
    #mask ve webcam çalıştır
        cv2.imshow('mask',mask)
        cv2.imshow('frame',frame)
    except:
        pass
     
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
    
cv2.destroyAllWindows()
vid.release()  

KeyboardInterrupt: 